## 整合Test代码

In [ ]:
import torch
import os
import numpy as np
from datasets.crowd import Crowd
from models.vgg_c import vgg19_trans
import argparse
import math

In [ ]:
class Args(object):
    def __init__(self):
        self.data_dir = ""
        self.save_dir = ""
        self.device = 0
args = Args()

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = args.device.strip()
datasets = Crowd(os.path.join(args.data_dir, 'test'), 512, 8, is_gray=False, method='val')
dataloader = torch.utils.data.DataLoader(datasets, 1, shuffle=False,
                                             num_workers=8, pin_memory=False)
device = torch.device('cuda')
model = vgg19_trans()
model.to(device)
model.eval()
model.load_state_dict(torch.load(args.save_dir, device))
epoch_minus = []
for inputs, count, name in dataloader:
    inputs = inputs.to(device)
    b, c, h, w = inputs.shape
    h, w = int(h), int(w)
    assert b == 1, 'the batch size should equal to 1 in validation mode'
    input_list = []
    if h >= 3584 or w >= 3584:
        h_stride = int(math.ceil(1.0 * h / 3584))
        w_stride = int(math.ceil(1.0 * w / 3584))
        h_step = h // h_stride
        w_step = w // w_stride
        for i in range(h_stride):
            for j in range(w_stride):
                h_start = i * h_step
                if i != h_stride - 1:
                    h_end = (i + 1) * h_step
                else:
                    h_end = h
                w_start = j * w_step
                if j != w_stride - 1:
                    w_end = (j + 1) * w_step
                else:
                    w_end = w
                input_list.append(inputs[:, :, h_start:h_end, w_start:w_end])
        with torch.set_grad_enabled(False):
            pre_count = 0.0
            for idx, input in enumerate(input_list):
                output = model(input)[0]
                pre_count += torch.sum(output)
        res = count[0].item() - pre_count.item()
        epoch_minus.append(res)
    else:
        with torch.set_grad_enabled(False):
            outputs = model(inputs)[0]
            res = count[0].item() - torch.sum(outputs).item()
            epoch_minus.append(res)